# MEPs in diseased vs healthy mice

In [ ]:
from tqdm import tqdm
from mepextract.extracting import Extractor
from scipy.signal import find_peaks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
import os
import pickle
import json

plt.style.use(['science', 'grid', 'custom', 'vibrant'])

## defining relevant folders and paths

In [ ]:
local = '/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test/MICE_EEG_2024 (93-257).csv'

temp = pd.read_csv(local)

temp2 = temp.iloc[:10]

temp2.to_csv('/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test/MICE_EEG_2024 (93-102).csv', index=False)

In [ ]:
test = '/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test/MICE_EEG_2024 (93-102).csv'
full = '/Volumes/STORAGE 1.0/UNIC Research/Motor Evoked Potentials/MICE_EEG_2024 (93-257).csv'

spreadsheet = pd.read_csv(test)

master_folder = '/Volumes/STORAGE 1.0/UNIC Research/Motor Evoked Potentials/MEPSSEP_raw_data/post 93'
test_folder = '/Users/fomo/Documents/Research/UNIC Research/Motor Evoked Potentials Test'

spreadsheet

In [70]:
spreadsheet['currentLevel'].mean()
spreadsheet['currentLevel'].std()

1.366764630635266

## extracting MEPs for all trials

In [62]:
# number of trials to iterate over
n = 10
sampling_rate = 30000

extracted = []

for i in tqdm(range(0, n), desc="Processing trials"):
    
    # relevant information
    trial = spreadsheet.session[i]
    notes = spreadsheet.loc[i]
    
    # defining extractor object
    extractor = Extractor(
        master_folder=test_folder , 
        trial=trial, 
        group=phenotype, 
        recording_channels=[5, 7],
        sampling_rate=sampling_rate)
    extractor.pre_stimulus = 300
    extractor.post_stimulus = 3000
    extractor.notes = notes
    
    # extracting relevant data
    extractor.extract_raw()
    extractor.extract_events(event_channel_number=13)
    extractor.get_event_data(export=False)
    
    detected = {'trial': trial, 'notes': notes, 'data': extractor.mep}

    extracted.append(detected)
    

Processing trials: 100%|██████████| 10/10 [00:03<00:00,  2.57it/s]


[{'trial': '2024-03-07_18-03-10',
  'notes': session                                      2024-03-07_18-03-10
  animalID                                                    A571
  cage                                                         NaN
  ear                                                          NaN
  phenotype                                                  5XFAD
  phenoCode                                                    2.0
  subrec                                                       1.0
  sessionClass                                                 MEP
  sessionType                                              sciatic
  currentLevel                                                1.49
  stim_lvl                                                     thr
  isoflo                                                       1.9
  trials_from                                                  1.0
  trials_to                                                     60
  notes           '

In [88]:
from scipy.ndimage import gaussian_filter1d as gf

# defining relevant channels, peak parameters, time axis and current level
time_in_ms = np.arange(-300, 3000) * (1/sampling_rate * 1000)
recording_channels = [5, 7]
search = {'height': (0, 10) ,'width': (1, 50), 'distance': 400}

current_mean = spreadsheet['currentLevel'].mean()
current_std = spreadsheet['currentLevel'].std() 

# temporary data
n = 10
fig, ax = plt.subplots(n, 1, figsize=(21, 7*n), dpi=210)

for i in range(n):
    
    temp = extracted[i]['data']
    current = extracted[i]['notes']['currentLevel']
    
    for channel in recording_channels:
        
        current = (current - current_mean)/current_std
        
        data = temp[channel, :, :]
        baseline = temp[channel, :10, :].mean(axis=1).mean(axis=0)
        correction = np.full((1, data.shape[0]), baseline)
        
        mean_events = (data.mean(axis=1)).flatten()
        std_events = (data.std(axis=1)).flatten()
        
        smoothed_mean = gf(mean_events, sigma=1.25)
        smoothed_std = gf(std_events, sigma=1.25)
        
        standardised = (1/current)*(smoothed_mean - smoothed_mean.mean())/smoothed_mean.std()
        
        pos_peaks, _ = find_peaks(standardised, height = search['height'], distance = search['distance'], width = search['width'])
        
        neg_peaks, _ = find_peaks(-standardised, height = search['height'], distance = search['distance'], width = search['width'])
    
        ax[i].plot(time_in_ms, standardised, label=f'Channel {channel}')
        ax[i].fill_between(time_in_ms, standardised - 1, standardised + 1, alpha=0.5)
    
        # Plot detected peaks
        ax[i].plot(time_in_ms[pos_peaks], standardised[pos_peaks], 'x', color='red')
        ax[i].plot(time_in_ms[neg_peaks], standardised[neg_peaks], 'x', color='blue')
        
        ax[i].set_title(f'Trial {extracted_peaks[i]["trial"]}')
        ax[i].set_ylim(-15, 15)
        ax[i].set_xlim(-2, 10)
        ax[i].set_xlabel('Time (ms)')
        ax[i].set_ylabel('Amplitude')
        ax[i].legend()
        
plt.tight_layout()
plt.show()
fig.savefig('/Users/fomo/Desktop/first_ten_trials_zone_one.png')